<a href="https://colab.research.google.com/github/diuliad/AED3/blob/main/AED3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
from time import time
from itertools import permutations
from scipy.sparse.csgraph import minimum_spanning_tree
import matplotlib.pyplot as plt
import numpy as np

In [11]:
from google.colab import files

uploaded = files.upload()

Saving tsp1_253.txt to tsp1_253 (1).txt
Saving tsp2_1248.txt to tsp2_1248 (1).txt
Saving tsp3_1194.txt to tsp3_1194 (1).txt
Saving tsp4_7013.txt to tsp4_7013 (1).txt
Saving tsp5_27603.txt to tsp5_27603 (1).txt


In [12]:
import os
os.listdir('/content')

['.config',
 'tsp3_1194.txt',
 'tsp5_27603.txt',
 'tsp3_1194 (1).txt',
 'tsp1_253 (1).txt',
 'tsp1_253.txt',
 'tsp2_1248.txt',
 'tsp5_27603 (1).txt',
 'tsp4_7013.txt',
 'tsp2_1248 (1).txt',
 'tsp4_7013 (1).txt',
 'sample_data']

In [13]:
# funcao pra ler a matriz de adjacencia do arquivo
def read_matrix_file(file):
    with open(file) as matrix_file:
        matrix = [list(map(int, line.split())) for line in matrix_file]
    return matrix

In [14]:
#funcao pra calcular o custo do caminho.
def calculate_path_cost(matrix, path):
    tsp_cost = 0
    nodes = range(len(matrix))
    for index in nodes:
        line = path[index]
        column = path[index + 1]
        edge_weight = matrix[line][column]
        tsp_cost += edge_weight
    return tsp_cost

In [15]:
#algoritmo aproximativo para o TSP usando MST
def approximate_tsp(matrix, initial_node=0):
    MST = minimum_spanning_tree(matrix).toarray().astype(int)
    nodes = range(len(MST))
    path = [initial_node]
    current_node = initial_node
    previous_node = -1

    while len(set(path)) != len(nodes):
        for connected_node in nodes:
            if MST[current_node, connected_node] == 0 and MST[connected_node, current_node] == 0:
                continue
            elif connected_node in path:
                continue
            else:
                path.append(connected_node)
                current_node = connected_node
                previous_node = -1
                break
        else:
            current_node = path[previous_node]
            previous_node -= 1

    path.append(initial_node)
    tsp_cost = calculate_path_cost(matrix, path)
    return tsp_cost, path

In [16]:
#algoritmo exato para o TSP com Brute force
from itertools import permutations

def brute_force_tsp(matrix):
    nodes = list(range(len(matrix)))
    min_cost = float("inf")
    min_path = []

    for perm in permutations(nodes):
        path = list(perm) + [perm[0]]
        cost = calculate_path_cost(matrix, path)
        if cost < min_cost:
            min_cost = cost
            min_path = path

    return min_cost, min_path

#func pra comparar os algoritmos e gerar os resultados
def compare_algorithms(matrix_file, run_brute_force=False):
    matrix = read_matrix_file(matrix_file)
    costs = dict()

In [17]:
#funcao para comparar os algoritmos
def compare_algorithms(matrix_file, run_brute_force=False):
    matrix = read_matrix_file(matrix_file)

    #teste com o algoritmo aproximativo
    costs = {}
    for initial_node in range(len(matrix)):
        start_time = time()
        cost, approximate_path = approximate_tsp(matrix, initial_node=initial_node)
        approximate_time = time() - start_time
        costs[cost] = {"path": approximate_path, "time": approximate_time}

    #melhor resultado do algoritmo aproximativo
    min_cost = min(costs.keys())
    min_path = costs[min_cost]["path"]
    min_time = costs[min_cost]["time"]

    #valor otimo
    file_name = matrix_file.split('/').pop()
    optimal_cost = int(file_name.split('_')[1].split('.')[0])

    #teste com o algoritmo exato
    bf_cost = "--"
    bf_time = "--"

    if run_brute_force:
        start_time = time()
        bf_cost, bf_path = brute_force_tsp(matrix)
        bf_time = time() - start_time

    return file_name, min_cost, min_time, optimal_cost, bf_cost, bf_time


In [ ]:
#listagem dos arquivos e configuração dos testes
files = [
    ("tsp1_253.txt", True),
    ("tsp2_1248.txt", True),
    ("tsp3_1194.txt", False),
    ("tsp4_7013.txt", False),
    ("tsp5_27603.txt", False)
]

#testando os arquivos
print("TSP\t\tAA Cost\t\tAA Time\t\tOptimal Cost\tBF Cost\t\tBF Time")
for tsp_file in files:
    tsp, brute_force = tsp_file
    tsp, ap_cost, ap_time, optimal_cost, bf_cost, bf_time = compare_algorithms(
        tsp, run_brute_force=brute_force
    )

    if brute_force:
        print(f'{tsp}\t\t{ap_cost}\t\t{ap_time:.5f}\t\t{optimal_cost}\t\t{bf_cost}\t\t{bf_time:.5f}')
    else:
        print(f'{tsp}\t\t{ap_cost}\t\t{ap_time:.5f}\t\t{optimal_cost}\t\t{bf_cost}\t\t{bf_time}')

